In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Memoire')

In [ ]:
import numpy as np
import astropy as ap
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from astropy.table import QTable

import read_mist_models

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import ToTensor

In [ ]:
def get_iso_data_panda(file):
    iso = read_mist_models.ISO(file)

    age = []
    logTeff = []
    # logL = []
    log_g = []
    mass = []
    for iso_ind in range(len(iso.isos)):
        age.extend(iso.isos[iso_ind]['log10_isochrone_age_yr'])
        logTeff.extend(iso.isos[iso_ind]['log_Teff'])
        # logL.extend(iso.isos[iso_ind]['log_L'])
        log_g.extend(iso.isos[iso_ind]['log_g'])
        mass.extend(iso.isos[iso_ind]['star_mass'])

    iso_data_all = QTable([age, logTeff, log_g, mass],
                        names=('age', 'logTeff', 'log_g', "mass"),
                        meta={'name': "iso data"})
    # print(iso_data_all.info)
    # print(iso_data_all)
    # display(iso_data_all.to_pandas())
    return iso_data_all.to_pandas()

In [ ]:
full_data = get_iso_data_panda("/content/drive/MyDrive/Memoire/data/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_p0.00_afe_p0.0_vvcrit0.0_basic.txt")

Reading in: /content/drive/MyDrive/Memoire/data/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_p0.00_afe_p0.0_vvcrit0.0_basic.txt


In [ ]:
display(full_data)

,age,logTeff,log_g,mass
0,5.0,3.468541,3.116651,0.100000
1,5.0,3.469176,3.114042,0.101392
2,5.0,3.471116,3.106133,0.105660
3,5.0,3.473035,3.098417,0.109915
4,5.0,3.474944,3.090875,0.114180
...,...,...,...,...
103984,10.3,4.370643,7.779334,0.528715
103985,10.3,4.363336,7.782175,0.528717
103986,10.3,4.356026,7.784962,0.528720
103987,10.3,4.348711,7.787695,0.528724


In [ ]:
no_massive_stars_data = full_data.where(full_data.mass < 5).dropna().reset_index(drop=True)

In [ ]:
display(no_massive_stars_data)

,age,logTeff,log_g,mass
0,5.0,3.468541,3.116651,0.100000
1,5.0,3.469176,3.114042,0.101392
2,5.0,3.471116,3.106133,0.105660
3,5.0,3.473035,3.098417,0.109915
4,5.0,3.474944,3.090875,0.114180
...,...,...,...,...
82360,10.3,4.370643,7.779334,0.528715
82361,10.3,4.363336,7.782175,0.528717
82362,10.3,4.356026,7.784962,0.528720
82363,10.3,4.348711,7.787695,0.528724


In [ ]:
no_massive_stars_data_2 = full_data.loc[full_data.mass < 5].reset_index(drop=True)

In [ ]:
display(no_massive_stars_data_2)

,age,logTeff,log_g,mass
0,5.0,3.468541,3.116651,0.100000
1,5.0,3.469176,3.114042,0.101392
2,5.0,3.471116,3.106133,0.105660
3,5.0,3.473035,3.098417,0.109915
4,5.0,3.474944,3.090875,0.114180
...,...,...,...,...
82360,10.3,4.370643,7.779334,0.528715
82361,10.3,4.363336,7.782175,0.528717
82362,10.3,4.356026,7.784962,0.528720
82363,10.3,4.348711,7.787695,0.528724


In [ ]:
# age_mass_no_massive_stars_data = no_massive_stars_data[["age", "mass"]] # pour choisir des colonnes

In [ ]:
print(no_massive_stars_data.shape)

(82365, 4)


In [ ]:
import random

# basic random train/test split
train_data = []
test_data = []
for index, row in no_massive_stars_data.iterrows():
    if random.randint(1, 100) <= 70: # enlever une row sur X? p/r au pas de l'âge
        train_data.append([row["age"], row["mass"], row["logTeff"], row["log_g"]])
    else:
        test_data.append([row["age"], row["mass"], row["logTeff"], row["log_g"]])

train_data_df = pd.DataFrame(train_data, columns=["age", "mass", "logTeff", "log_g"])
test_data_df = pd.DataFrame(test_data, columns=["age", "mass", "logTeff", "log_g"])

display(train_data_df)
display(test_data_df)

,age,mass,logTeff,log_g
0,5.0,0.100000,3.468541,3.116651
1,5.0,0.101392,3.469176,3.114042
2,5.0,0.105660,3.471116,3.106133
3,5.0,0.109915,3.473035,3.098417
4,5.0,0.114180,3.474944,3.090875
...,...,...,...,...
57679,10.3,0.528708,4.392544,7.770538
57680,10.3,0.528710,4.385247,7.773515
57681,10.3,0.528712,4.377947,7.776449
57682,10.3,0.528717,4.363336,7.782175


,age,mass,logTeff,log_g
0,5.0,0.126952,3.480566,3.069404
1,5.0,0.139721,3.486031,3.049569
2,5.0,0.143969,3.487810,3.043357
3,5.0,0.159488,3.494046,3.027242
4,5.0,0.165534,3.496405,3.022706
...,...,...,...,...
24676,10.3,0.528697,4.457995,7.741572
24677,10.3,0.528702,4.421690,7.758184
24678,10.3,0.528715,4.370643,7.779334
24679,10.3,0.528720,4.356026,7.784962


In [ ]:
from torch.utils.data import Dataset, DataLoader

class Custom_Dataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return self.data.shape[0]
    def __getitem__(self, ind):
        row = self.data.iloc[ind]
        x = torch.tensor([row["age"], row["mass"]])
        y = torch.tensor([row["logTeff"], row["log_g"]])
        return x, y

train_set = Custom_Dataset(train_data_df)
test_set  = Custom_Dataset(test_data_df)

batch_size = 64
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_set,  batch_size=batch_size, shuffle=True)

In [ ]:
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()

      self.fc1 = nn.Linear(2, 50)
      self.fc2 = nn.Linear(50, 50)
      self.fc3 = nn.Linear(50, 50)
      self.fc4 = nn.Linear(50, 2)
      self.double()
      # changes the type of the nn.Linear functions to double to match the type of the data

    # x represents our data
    def forward(self, x):
      x = self.fc1(x)
      x = F.relu(x)

      x = self.fc2(x)
      x = F.relu(x)

      x = self.fc3(x)
      x = F.relu(x)

      output = self.fc4(x)
      return output

# test_nn = Net()
# print(test_nn)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            # print(pred.argmax(1))
            # print(y.T.shape)
            correct += (pred.argmax(1) == y.T).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
device = ("cuda" if torch.cuda.is_available()
          else "mps" if torch.backends.mps.is_available()
          else "cpu")

model = Net().to(device)

learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loop(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 4.714708  [   64/57684]
loss: 4.261964  [ 6464/57684]
loss: 4.092833  [12864/57684]
loss: 3.720919  [19264/57684]
loss: 4.161231  [25664/57684]
loss: 3.955437  [32064/57684]
loss: 4.005818  [38464/57684]
loss: 4.541135  [44864/57684]
loss: 4.119992  [51264/57684]
loss: 4.038238  [57664/57684]
Test Error: 
 Accuracy: 0.0%, Avg loss: 3.899088 

Epoch 2
-------------------------------
loss: 4.139632  [   64/57684]
loss: 3.496333  [ 6464/57684]
loss: 3.981110  [12864/57684]
loss: 3.425180  [19264/57684]
loss: 3.517272  [25664/57684]
loss: 3.455810  [32064/57684]
loss: 3.175687  [38464/57684]
loss: 3.633249  [44864/57684]
loss: 3.734791  [51264/57684]
loss: 4.202021  [57664/57684]
Test Error: 
 Accuracy: 0.0%, Avg loss: 3.877768 

Epoch 3
-------------------------------
loss: 4.117396  [   64/57684]
loss: 4.023086  [ 6464/57684]
loss: 3.586623  [12864/57684]
loss: 3.821555  [19264/57684]
loss: 3.969506  [25664/57684]
loss: 3.539190  [32064/57684

In [ ]:
device = ("cuda" if torch.cuda.is_available()
          else "mps" if torch.backends.mps.is_available()
          else "cpu")

model = Net().to(device)

# learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loop(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 4.461050  [   64/57684]
loss: 4.384904  [ 6464/57684]
loss: 3.618896  [12864/57684]
loss: 3.453882  [19264/57684]
loss: 4.084842  [25664/57684]
loss: 4.182710  [32064/57684]
loss: 4.229862  [38464/57684]
loss: 4.198112  [44864/57684]
loss: 4.318182  [51264/57684]
loss: 4.127508  [57664/57684]
Test Error: 
 Accuracy: 0.0%, Avg loss: 3.890052 

Epoch 2
-------------------------------
loss: 4.445967  [   64/57684]
loss: 4.137360  [ 6464/57684]
loss: 3.986116  [12864/57684]
loss: 3.803878  [19264/57684]
loss: 3.739218  [25664/57684]
loss: 4.158844  [32064/57684]
loss: 4.258431  [38464/57684]
loss: 3.722002  [44864/57684]
loss: 4.356527  [51264/57684]
loss: 3.631801  [57664/57684]
Test Error: 
 Accuracy: 0.0%, Avg loss: 3.873498 

Epoch 3
-------------------------------
loss: 3.857444  [   64/57684]
loss: 4.144054  [ 6464/57684]
loss: 4.274252  [12864/57684]
loss: 3.809432  [19264/57684]
loss: 3.822750  [25664/57684]
loss: 3.954130  [32064/57684

In [ ]:
device = ("cuda" if torch.cuda.is_available()
          else "mps" if torch.backends.mps.is_available()
          else "cpu")

model = Net().to(device)

# learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.MSELoss()

epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loop(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 13.232842  [   64/57684]
loss: 3.395629  [ 6464/57684]
loss: 3.514803  [12864/57684]
loss: 2.909109  [19264/57684]
loss: 2.413548  [25664/57684]
loss: 2.699156  [32064/57684]
loss: 2.660697  [38464/57684]
loss: 3.448908  [44864/57684]
loss: 2.342431  [51264/57684]
loss: 2.225720  [57664/57684]
Test Error: 
 Accuracy: 0.0%, Avg loss: 2.239587 

Epoch 2
-------------------------------
loss: 2.287695  [   64/57684]
loss: 2.293644  [ 6464/57684]
loss: 2.552396  [12864/57684]
loss: 2.395093  [19264/57684]
loss: 2.235809  [25664/57684]
loss: 2.062237  [32064/57684]
loss: 2.350491  [38464/57684]
loss: 1.857315  [44864/57684]
loss: 1.845477  [51264/57684]
loss: 1.738508  [57664/57684]
Test Error: 
 Accuracy: 0.0%, Avg loss: 2.069988 

Epoch 3
-------------------------------
loss: 2.633609  [   64/57684]
loss: 2.230633  [ 6464/57684]
loss: 1.450600  [12864/57684]
loss: 1.870011  [19264/57684]
loss: 1.397055  [25664/57684]
loss: 1.256015  [32064/5768

In [ ]:
model.eval()

Net(
  (fc1): Linear(in_features=2, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=2, bias=True)
)

In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
print(model(torch.tensor([np.float64(10.3),	np.float64(0.528708)]).to(device)))

tensor([4.1771, 5.0025], device='cuda:0', dtype=torch.float64,
       grad_fn=<ViewBackward0>)
